In [ ]:
#! /usr/bin/env python3

In [ ]:
def no_argument():
    print("I heartily agree!")
    return

In [ ]:
no_argument()

### a working example

In [ ]:
from tempfile import mkstemp
import os
from pathlib import Path
from contextlib import contextmanager

@contextmanager
def tempfile():
    handle, name = mkstemp()    # returns open handle and filename
    os.close(handle)            # don't need open handle
    file = Path(name)           # like Paths rather than raw strings
    yield file                  # return the Path
    file.unlink()               # delete the temporary file

In [ ]:
with tempfile() as test:
    print(f"{test=}")
    save = test
print(f"{save.exists()}")

## a function with some arguments

In [ ]:
def some_arguments(foo, bar, baz):
    print(f"{foo=}, {bar=}, {baz=} => '{foo} {bar} {baz}'")
    return

In [ ]:
some_arguments("yes", "but", "what about...")

In [ ]:
some_arguments()

### a working example

In [ ]:
@contextmanager
def tempfile(prefix, suffix, directory):
    handle, name = mkstemp(suffix, prefix, directory)
    os.close(handle)
    file = Path(name)
    yield file
    file.unlink()
    return

In [ ]:
with tempfile() as test:
    print(f"{test=}")
    save = test

## a function with some arguments with defaults

In [ ]:
def defaults(foo="Hello", baz="World!"):
    print(f"{foo=}, {baz=} => '{foo}, {baz}'")
    return

In [ ]:
defaults()                  # Hello, World!

In [ ]:
defaults(foo="Greetings")   # Greetings, World!

### a working example

In [ ]:
@contextmanager
def tempfile(prefix="output-", suffix=".txt", directory=None):
    handle, name = mkstemp(suffix, prefix, directory)
    os.close(handle)
    file = Path(name)
    yield file
    file.unlink()
    return

In [ ]:
with tempfile() as test:
    print(f"{test=}")
    save = test

## a function with some arguments, with _some_ defaults

In [ ]:
def some_defaults(foo, bar, baz="World!"):
    print(f"{foo=}, {bar=}, {baz=} => '{foo}{bar}{baz}'")
    return

In [ ]:
try:
    # Not allowed, must specify `foo` and `bar`.
    some_defaults()
except TypeError as e:
    print(e)

In [ ]:
some_defaults("Aloha", ": ")    # Aloha: World!

In [ ]:
def wont_work(foo="abracadabra", bar, baz):
    return

### a working example

In [ ]:
@contextmanager
def tempfile(prefix, suffix, directory=None):
    handle, name = mkstemp(suffix, prefix, directory)
    os.close(handle)
    file = Path(name)
    yield file
    file.unlink()
    return

In [ ]:
with tempfile("some-output-", ".bin") as test:
    print(f"{test=}")
    save = test

## another function with defaults

In [ ]:
def welcome(greet="Hello", sep=", ", entity="World", stop="!"):
    print(f"{greet=}, {sep=}, {entity=}, {stop=} => '{greet}{sep}{entity}{stop}'")
    return

In [ ]:
welcome(entity="everyone", sep=": ", stop=".", greet="Entertaining")    # Entertaining: everyone.

In [ ]:
welcome("Salutations", "/", "IDM", "#")

----

## Positional-Only Parameters
[PEP 570 -- Python Positional-Only Parameters](https://www.python.org/dev/peps/pep-0570/)

With positional-or-keyword parameters, the mix of calling conventions is not always desirable. Authors may want to restrict usage of an API by disallowing calling the API with keyword arguments, which exposes the name of the parameter when part of the public API.

In [ ]:
def positional_only(greet="Hello", entity="World", /, sep=", ", stop="!"):
    print(f"{greet=}, {sep=}, {entity=}, {stop=} => '{greet}{sep}{entity}{stop}'")
    return

In [ ]:
try:
    # Not allowed`greet` and `entity` are positional only.
    positional_only(entity="everyone", sep=": ", stop=".", greet="Entertaining")
except TypeError as e:
    print(e)

In [ ]:
positional_only("Welcome", "co-workers")    # Welcome, co-workers!

### Name of Parameter as Part of Public API

In [ ]:
def intdiv(foo, bar):
    return foo // bar

In [ ]:
intdiv(foo=42, bar=21)

In [ ]:
def intdiv(numerator, denominator):
    return numerator // denominator

In [ ]:
intdiv(foo=42, bar=21)

In [ ]:
def intdiv(numerator, denominator, /):
    return numerator // denominator

In [ ]:
intdiv(numerator=42, bar=21)

In [ ]:
intdiv(42, 21)

## *args == _star_ args, arbitrary arguments gathered into a tuple

In [ ]:
def star_args(*args):
    print(f"'{args[0]}'...'{args[-1]}'") if len(args) else None
    print(f"{type(args)} {args=}")
    return

In [ ]:
star_args("acts", "cast", "cats", "scat")   # arguments gathered into a tuple...

In [ ]:
t = ("Institute", "for", "Disease", "Modeling")
star_args(t)

In [ ]:
star_args(*t)

### an example

In [ ]:
def bigsigma(*args):
    return sum(args)

In [ ]:
bigsigma(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [ ]:
bigsigma(*range(101))

## **kwargs == _keyword_ args, arbitrary arguments specified as key:value pairs gathered into a dictionary

In [ ]:
def keyword_args(**kwargs):
    print(f"{kwargs['foo']=}") if "foo" in kwargs else "No 'foo' found. :^("
    print(f"{type(kwargs)} {kwargs=}")
    return

In [ ]:
keyword_args(a="acts", b="cast", foo="cats", bar="scat")    # arguments gathered into a dictionary

In [ ]:
d = {"name": "Bill & Melinda Gates Foundation", "street": "500 Fifth Ave N", "city": "Seattle", "state": "WA", "zipcode": "98109-4636"}
d

In [ ]:
try:
    # Not allowed, need at least one keyword argument to go into kwargs.
    keyword_args(d)
except TypeError as e:
    print(e)

In [ ]:
keyword_args(**d)

### an example

In [ ]:
import json
def demographics_node(latitude, longitude, population, **kwargs):
    dictionary = {"latitude": latitude, "longitude": longitude, "population": population }
    dictionary.update(kwargs)
    return json.dumps(dictionary)

In [ ]:
base = demographics_node(47.623294, -122.346503, 1500)
print(base)
node = demographics_node(47.623294, -122.346503, 1500, **{"name": "bmgf", "airport": False})
print(node)

----

## Keyword-Only Arguments

[PEP 3102 -- Keyword-Only Arguments](https://www.python.org/dev/peps/pep-3102/)

There are often cases where it is desirable for a function to take a variable number of arguments. The Python language supports this using the 'varargs' syntax (*name), which specifies that any 'left over' arguments be passed into the varargs parameter as a tuple.

One limitation on this is that currently, all of the regular argument slots must be filled before the vararg slot can be.

This is not always desirable. One can easily envision a function which takes a variable number of arguments, but also takes one or more 'options' in the form of keyword arguments. Currently, the only way to do this is to define both a varargs argument, and a 'keywords' argument (**kwargs), and then manually extract the desired keywords from the dictionary.

In [ ]:
def keyword_only(greet="Hello", entity="World", *, sep=", ", stop="!"):
    print(f"{greet=}, {sep=}, {entity=}, {stop=} => '{greet}{sep}{entity}{stop}'")
    return

In [ ]:
keyword_only("Aloha", "Everyone", sep=": ", stop="$")

In [ ]:
try:
    # Not allowed, `sep` and `stop` must be specified by keyword.
    keyword_only("Guten Tag", "Jeder", "! ¡", "!")
except TypeError as e:
    print(e)

In [ ]:
keyword_only("Guten Tag", "Jeder", sep="! ¡", stop="!")

### an example?

In [ ]:
from functools import reduce
def product(values):
    return reduce(lambda x, y: x*y, values, 1)
    
def aggregate(a, b, *args, multiply=False):
    print(f"{a=}, {b=}, {args=}")
    return a*b*product(args) if multiply else a+b+sum(args)

In [ ]:
aggregate(*range(1, 5))

In [ ]:
aggregate(13, 42, True)

In [ ]:
aggregate(*range(1,5), multiply=True)

In [ ]:
def expect(test, expected, exact=True, epsilon=2**-20):
    return test == expected if exact else abs(test - expected) <= epsilon

In [ ]:
from math import pi
expect(22/7, pi)

In [ ]:
expect(22/7, pi, False, 0.002)

In [ ]:
def expect(test, expected, *, exact=True, epsilon=2**-20):
    return test == expected if exact else abs(test - expected) <= epsilon

In [ ]:
expect(22/7, pi, False, 0.002)

In [ ]:
expect(22/7, pi, exact=False, epsilon=0.002)

## Let's Put 'em (Almost) All Together

In [ ]:
def almost_everything(a, b, c, /, d, e, f, *toople, **kwargs):
    print(f"{a=}")
    print(f"{b=}")
    print(f"{c=}")
    print(f"{d=}")
    print(f"{e=}")
    print(f"{f=}")
    print(f"{toople=}")
    print(f"{kwargs=}")
    return

In [ ]:
almost_everything(3, 1, 4, 1, 5, 9, "foo", "bar", "baz", fname="Christopher", lname="Lorton")

In [ ]:
almost_everything(3, 1, 4, d=1, e=5, f=9)

In [ ]:
almost_everything(3, 1, 4, d=1, e=5, f=9, "foo", "bar", "baz")

In [ ]:
almost_everything(3, 1, 4, "foo", "bar", "baz", d=1, e=5, f=9)

In [ ]:
almost_everything(3, 1, 4, 1, 5, 9, "foo", "bar", "baz")

In [ ]:
almost_everything(3, 1, 4, 1, 5, 9, "foo", bar="bar", "baz")

## Another Try at (Almost) Everything...

In [ ]:
def test(a, b, c, /, *toople, d, e, f, **kwargs):
    print(f"{a=}")
    print(f"{b=}")
    print(f"{c=}")
    print(f"{d=}")
    print(f"{e=}")
    print(f"{f=}")
    print(f"{toople=}")
    print(f"{kwargs=}")
    return

In [ ]:
test(3, 1, 4, "foo", "bar", "baz", d=1, e=5, f=9, lname="Einstein", fname="Albert")

## ¡Don't Use Mutable Values for Defaults!

In [ ]:
class Thing(object):
    def __init__(self, values=[]):
        self._values = values
        return
    
    @property
    def values(self):
        return self._values

In [ ]:
thing1 = Thing()
thing1.values.extend([ 3, 1, 4, 1, 5, 9 ])
print(f"{thing1.values}")

In [ ]:
thing2 = Thing()
print(f"{thing2.values}")

In [ ]:
thing2.values.extend([0, 1, 1, 2, 3, 5])
thing2.values

In [ ]:
thing1.values

In [ ]:
class BetterThing(object):
    def __init__(self, values=None):
        self._values = values if values else []
        return
    
    @property
    def values(self):
        return self._values

In [ ]:
bthing = BetterThing()
bthing._values.extend([ 0, 1, 1, 2, 3, 5])
print(f"{bthing.values}")

In [ ]:
another_bthing = BetterThing()
print(f"{another_bthing.values}")

## args and kwargs

In [ ]:
def wrapit(fn):
    def wrapper(*args, **kwargs):
        print(f"{args=}")
        print(f"{kwargs=}")
        return fn(*args, **kwargs)
    return wrapper

In [ ]:
def add_three(arg):
    return arg + 3

In [ ]:
add_three(7)

In [ ]:
@wrapit
def add_five(arg, **kwargs):
    print(f"Didn't use '{kwargs}'")
    return arg + 5

In [ ]:
add_five(37)

In [ ]:
add_five(37, option=False)